In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("trainn.csv")

In [4]:
df.head()

,title,text,subject,date,real
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [5]:
df.describe()

,real
count,44856.000000
mean,0.522539
std,0.499497
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44856 entries, 0 to 44855
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44856 non-null  object
 1   text     44856 non-null  object
 2   subject  44856 non-null  object
 3   date     44856 non-null  object
 4   real     44856 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [7]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
real       0
dtype: int64

In [8]:
df=df.fillna('')

In [9]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
real       0
dtype: int64

In [10]:
df.columns

Index(['title', 'text', 'subject', 'date', 'real'], dtype='object')

In [11]:
df=df.drop([ 'subject', 'date'], axis=1)

In [12]:
df.head()

,title,text,real
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,0


In [13]:
from nltk.corpus import stopwords

In [14]:
from nltk.stem.porter import PorterStemmer

In [15]:
import re

In [16]:
port_stem=PorterStemmer()

In [17]:
port_stem

<PorterStemmer>

In [19]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [22]:
df['title'] = df['title'].apply(stemming) + " " + df['text'].apply(stemming)


In [23]:
x=df['title']

In [24]:
y=df['real']

In [25]:
x.shape

(44856,)

In [26]:
y.shape

(44856,)

In [210]:
from sklearn.model_selection import train_test_split

In [211]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [212]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [213]:
vect=TfidfVectorizer()

In [214]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [215]:
x_test.shape

(8972, 80678)

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
model = LogisticRegression(max_iter=200)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
prediction=model.predict(x_test)

In [ ]:
prediction

In [ ]:
model.score(x_test, y_test)

In [ ]:
import pickle

In [ ]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

In [ ]:

pickle.dump(y_test, open('y_test.pkl', 'wb'))
pickle.dump(x_test, open('x_test_vect.pkl', 'wb'))


In [6]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

NameError: name 'pickle' is not defined

In [7]:
load_model=pickle.load(open('model.pkl', 'rb'))

NameError: name 'pickle' is not defined

In [8]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [9]:
val=fake_news(
    """
	Dr Urjit Patel, the former Reserve Bank of India governor, has been appointed Executive Director (ED) representing India at the International Monetary Fund after the Centre's approval for a three-year tenure.

"The Appointments Committee of the Cabinet has approved the appointment of Dr. Urjit Patel, Economist and Former RBI Governor, to the post of Executive Director (ED) at the International Monetary Fund, for a period of three years with effect from the date of assumption of charge of the post, or until further orders, whichever is earlier," said the government in a press release.
"""
)

NameError: name 'stemming' is not defined

In [230]:
if val==[0]:
    print('reliable')
else:
    print('unreliable')

reliable


In [232]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, prediction))
print("Precision:", precision_score(y_test, prediction))
print("Recall:", recall_score(y_test, prediction)) 
print("F1 Score:", f1_score(y_test, prediction)) 


Accuracy: 0.9865135978600089
Precision: 0.9882050182286082
Recall: 0.9858793324775353
F1 Score: 0.9870408053978794


In [ ]:
# DL
